In [ ]:
import time
from tqdm import tqdm_notebook as tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

sns.set_style("whitegrid")
rcParams.update({'figure.autolayout': True})

In [ ]:
idx = pd.IndexSlice

In [ ]:
df = pd.read_excel("../data/kemija loša.xlsx", decimal = ",", thousands = ".")
df.head()

In [ ]:
df.loc[:, "Vodni tip"] = df.loc[:, "Vodni tip"].astype(str)
bioloskiParametri = list(df.columns[-3:])
kemijskiParametri = list(df.columns[1:-3])

## Opisne statistike

Izbacujemo mulitIndex i radimo samo sa srednjom vrijednosti

In [ ]:
frame = df.loc[idx[:, "SR.VR."], :].reset_index().drop(["level_0", "level_1"], axis = 1)
frame.head()

In [ ]:
frame.info()

Postoji problem s električnom vodljivosti - castana je u object.<br>
Problem je u reprezentaciji broja npr. 1.290,75 - rješeno dodavanjem thousands = "." u read_excel

In [ ]:
frame.loc[:, kemijskiParametri].head()

In [ ]:
frame.loc[:, bioloskiParametri].head()

In [ ]:
vodniTipovi = list(frame.loc[:, "Vodni tip"].unique())
vodniTipovi, len(vodniTipovi)

In [ ]:
def toTableIndex(index, nRow = 3, nCol = 4):
    row = index // nCol
    if row == 0:
        col = index
    else:
        col = (index % nCol) 
    return (row, col)

In [ ]:
bigFig, axes = plt.subplots(3, 4, figsize = (50, 35))

for num, tip in enumerate(vodniTipovi):
    
    tmpFrame = frame.loc[frame.loc[:, "Vodni tip"] == tip]
    ax = axes[toTableIndex(num)[0], toTableIndex(num)[1]]
    
    sns.heatmap(tmpFrame.corr().applymap(lambda x: "{:.2f}".format(x)).astype(float), cmap = "inferno", ax = ax, annot = True)
    ax.set_title("Vodni tip: {}".format(tip))
    print("Checkpoint: {}".format(num))

bigFig.savefig("../pics/corr/corrPlot_AllInOne.pdf")
plt.show(bigFig)

In [ ]:
korelacijePoTipovima = dict()
for tip in ([None] + vodniTipovi):
    if tip is not None:
        tmpFrame = frame.loc[frame.loc[:, "Vodni tip"] == tip]
    else:
        tmpFrame = frame
        tip = "Svi"
        
    fig, ax = plt.subplots(figsize = (15, 8))
    tmpCorr =  tmpFrame.corr().applymap(lambda x: "{:.2f}".format(x)).astype(float)
    korelacijePoTipovima[tip] = tmpCorr.copy()
    sns.heatmap(tmpCorr, cmap = "inferno", ax = ax, annot = True)
    ax.set_title("Vodni tip: {}".format(tip))
    fig.savefig("../pics/corr/corrPlot_{}.pdf".format(tip))
    plt.show(fig)

keys = list(korelacijePoTipovima.keys())
for i in range(len(keys)):
    for j in range(i + 1, len(keys)):
        fig, ax = plt.subplots(figsize = (15, 8))
        key1, key2 = keys[i], keys[j]
        df1, df2 = korelacijePoTipovima[key1], korelacijePoTipovima[key2]
        diffFrame = (df2 - df1).abs()
        sns.heatmap(diffFrame, cmap = "inferno", ax = ax, annot = True)
        ax.set_title("corr |{} - {}|".format(key1, key2))
        fig.savefig("../pics/corrDiff/corrPlotDiff_{}_{}.pdf".format(key1, key2))
        plt.show(fig)

Radimo permutacijski test kako bi odredili značajnost korelacija

In [ ]:
def singlePerm(twoValFrame, nPerm):
    permArray = np.zeros(nPerm)
    for i in range(nPerm):
        col1 = list(twoValFrame.columns)[1]
        twoValFrame.loc[:, col1] = np.random.permutation(twoValFrame.loc[:, col1].values)
        permArray[i] = twoValFrame.corr().values[0, 1]
    return permArray.copy()
    

def corrPerm(dFrame, nPerm = 5_000):
    corrFrame = pd.DataFrame()
    cols = list(dFrame.columns)
    for i in range(len(cols)):
        for j in range(i + 1, len(cols)):
            corrFrame.loc[:, "corr({} ,{})".format(cols[i], cols[j])] = singlePerm(
                dFrame.loc[:, [cols[i], cols[j]]], nPerm)
    return corrFrame

In [ ]:
alpha = 0.05
for tip in tqdm([None] + vodniTipovi):
    if tip is not None:
        tmpFrame = frame.loc[frame.loc[:, "Vodni tip"] == tip].drop(["Vodni tip"], axis = 1)
    else:
        tmpFrame = frame.drop(["Vodni tip"], axis = 1)
        tip = "Svi"
        
    fig, axes = plt.subplots(2, 1, figsize = (15, 16))
    fig2, axes2 = plt.subplots(2, 1, figsize = (15, 16))
    fig3, ax3 = plt.subplots(figsize = (15, 8))
    dropFig, dropAx = plt.subplots(figsize = (15, 8))
    
    tmpCorr =  tmpFrame.corr().applymap(lambda x: "{:.2f}".format(x)).astype(float)
    pVals = tmpCorr.copy()
    signif = tmpCorr.copy()
    features = list(tmpCorr.columns)
    for i in tqdm(range(len(features)), leave = False):
        for j in tqdm(range(i + 1, len(features)), leave = False):
            corrVal = tmpCorr.loc[features[i], features[j]]
            permSamples = corrPerm(tmpFrame.loc[:, [features[i], features[j]]])
            if np.isnan(corrVal):
                pVal = 1.0
            elif corrVal >= 0:
                pVal = (permSamples > corrVal).mean()[0]
            else:
                pVal = (permSamples < corrVal).mean()[0]
            pVals.loc[features[i], features[j]] = pVal
            pVals.loc[features[j], features[i]] = pVal
            signif.loc[features[i], features[j]] = float(pVal < alpha)
            signif.loc[features[j], features[i]] = float(pVal < alpha)
            
    sns.heatmap(tmpCorr, cmap = "inferno", ax = axes[0], annot = True)
    sns.heatmap(pVals, cmap = "inferno", ax = axes[1], annot = True)
    
    sns.heatmap(tmpCorr, cmap = "inferno", ax = axes2[0], annot = True)
    sns.heatmap(signif, cmap = "inferno", ax = axes2[1], annot = True)
    
    tmpCorr.fillna(0, inplace = True)
    signif.fillna(0, inplace = True)
    
    sns.heatmap(tmpCorr, cmap = "inferno", ax = dropAx, annot = True)
    sns.heatmap(signif, cmap = "inferno", ax = ax3, annot = True)
    
    
    
    
    def corrTrans(value):
        if value == "250.0":
            return " "
        else:
            return value
    
    for t1, t2 in zip(ax3.texts, dropAx.texts):
        t1.set_text(corrTrans(t2.get_text()))
    
    axes[0].set_title("Vodni tip: {}".format(tip))
    axes[1].set_title("p-vrijednost")
    fig.savefig("../pics/corrPermTest/corrPlot_{}.pdf".format(tip))
    plt.show(fig)
    
    axes2[0].set_title("Vodni tip: {}".format(tip))
    axes2[1].set_title("značajnost")
    fig2.savefig("../pics/corrPermTest/corrPlotsignif_{}.pdf".format(tip))
    plt.show(fig2)
    
    ax3.set_title("Vodni tip: {}".format(tip))
    fig3.savefig("../pics/corrMerge/corrPlot_{}.pdf".format(tip))
    plt.show(fig3)
    plt.show(dropFig)